In [48]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
import keras

In [11]:
class PerformanceReport():

    def __init__(self):

        self.report = pd.DataFrame(columns=[ # We initialize our report DataFrame
            'Accuracy',
            'Precision_macro',
            'Precision_micro',
            'Recall_macro',
            'Recall_micro',
            'F1_macro',
            'F1_micro'])

    def evaluate(self, name:str ,y_true:np.array ,y_pred:np.array):
 
        accuracy = accuracy_score(y_true, y_pred) # We calculate the scores
        precision_macro = precision_score(y_true, y_pred, average='macro')
        precision_micro = precision_score(y_true, y_pred, average='micro')
        recall_macro = recall_score(y_true, y_pred, average='macro')
        recall_micro = recall_score(y_true, y_pred, average='micro')
        f1_macro = f1_score(y_true, y_pred, average='macro')
        f1_micro = f1_score(y_true, y_pred, average='micro')

        self.report.loc[name,:] = [ # We append the scores to our report
            accuracy,
            precision_macro,
            precision_micro,
            recall_macro,
            recall_micro,
            f1_macro,
            f1_micro]
            
        print(self.report.loc[name,:])
    
    def get(self):

        return self.report
    
report = PerformanceReport()

In [12]:
# X_train = pd.read_csv('./X_train.csv',index_col=0)
# X_test = pd.read_csv('./X_test.csv',index_col=0)
# y_train = pd.read_csv('./y_train.csv',index_col=0)
# y_test = pd.read_csv('./y_test.csv',index_col=0)

In [67]:
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(32, input_dim=X_train.shape[1], activation='tanh'))
	model.add(Dropout(0.8))
	model.add(Dense(16, activation='tanh'))
	# model.add(Dense(8, input_dim=X_train.shape[1], activation='relu'))
	model.add(Dense(y_train.shape[1], activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [68]:
model = baseline_model()
model.fit(X_train,y_train,epochs=50, batch_size=1024, verbose=1, validation_data=(X_test,y_test))

Epoch 1/50


2022-02-15 01:03:01.649594: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


39/39 [==============================] - 1s 25ms/step - loss: 20.1787 - accuracy: 0.0228 - val_loss: 19.8269 - val_accuracy: 0.0159
Epoch 2/50
 1/39 [..............................] - ETA: 0s - loss: 20.2386 - accuracy: 0.0312

2022-02-15 01:03:02.660869: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


39/39 [==============================] - 1s 19ms/step - loss: 19.5460 - accuracy: 0.0255 - val_loss: 18.8680 - val_accuracy: 0.0159
Epoch 3/50
39/39 [==============================] - 1s 17ms/step - loss: 18.8473 - accuracy: 0.0369 - val_loss: 18.4306 - val_accuracy: 0.0872
Epoch 4/50
39/39 [==============================] - 1s 18ms/step - loss: 18.5088 - accuracy: 0.1437 - val_loss: 18.2400 - val_accuracy: 0.1835
Epoch 5/50
39/39 [==============================] - 1s 18ms/step - loss: 18.3328 - accuracy: 0.1806 - val_loss: 18.1222 - val_accuracy: 0.1835
Epoch 6/50
39/39 [==============================] - 1s 18ms/step - loss: 18.2200 - accuracy: 0.1816 - val_loss: 18.0367 - val_accuracy: 0.1835
Epoch 7/50
39/39 [==============================] - 1s 19ms/step - loss: 18.1436 - accuracy: 0.1816 - val_loss: 17.9747 - val_accuracy: 0.1835
Epoch 8/50
39/39 [==============================] - 1s 17ms/step - loss: 18.0862 - accuracy: 0.1816 - val_loss: 17.9275 - val_accuracy: 0.1835
Epoch 9/50

KeyboardInterrupt: 

In [51]:
model = keras.Sequential()
model.add(keras.layers.Dense(1000, kernel_initializer=keras.initializers.he_normal(seed=1), activation='relu', input_dim=X_train.shape[1]))
model.add(keras.layers.Dropout(0.81))
model.add(keras.layers.Dense(1000, kernel_initializer=keras.initializers.he_normal(seed=2), activation='relu'))
model.add(keras.layers.Dropout(0.81))
model.add(keras.layers.Dense(y_train.shape[1], kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=4), activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
# estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [15]:
# Check for TensorFlow GPU access
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [17]:
preds = model.predict(X_test)

2022-02-15 00:46:54.690522: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [18]:
preds

array([[9.11187902e-02, 6.99506626e-02, 6.09804057e-02, ...,
        3.60428479e-31, 7.22908895e-33, 5.42035609e-32],
       [1.34521872e-01, 9.66031253e-02, 8.00159499e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.46843687e-01, 1.03459403e-01, 8.44333768e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.89140265e-02, 4.05406319e-02, 3.77206244e-02, ...,
        1.37905771e-21, 1.02468715e-22, 3.87162355e-22],
       [8.59614834e-02, 6.63711727e-02, 5.83724976e-02, ...,
        4.05694698e-30, 9.45740865e-32, 6.48310525e-31],
       [1.13306820e-01, 8.40279162e-02, 7.12787583e-02, ...,
        1.69155560e-35, 1.94300592e-37, 1.93178980e-36]], dtype=float32)